In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install notebook

In [ ]:
pip install bs4

In [ ]:
import requests

In [ ]:
pip install selenium

In [ ]:
#selenium 사용
    #  https://chromedriver.chromium.org/downloads 접속 후 크롬 드라이버 다운 (내 크롬과 같은 버전으로 다운로드)
    #  크롬 정보 보기 |  크롬 실행 -> 설정(우측 위 점3개 클릭 후 설정) -> 크롬 정보
    #  다운 후 압축풀기 -> jupyter notebook 홈에서 exe 파일 업로드 (파이썬 파일과 크롬 드라이버 경로가 같아야 함)

In [ ]:
# 1.3
# 썸네일 웹 -> 모바일 변경버전 (1.1)
# 파일 저장시 ? / 등 특수문자 저장 안되는 거 수정 완료 (1.2)
    # ? 같은 특수문자를 제거하고 파일을 저장 -> 제거 안하고 저장하는 방법 or 저장 후 다시 특수문자 붙이기(os모듈)
# 자동 로그인 추가 (1.3)
import time
import getpass
from selenium.webdriver.common.keys import Keys

from selenium import webdriver
 #-*- coding: UTF-8 -*-
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait

print("아이디를 입력해주세요")
kakao_id = input()
print("비밀번호를 입력해주세요")
kakao_password = getpass.getpass()

print()


chromedriver = './chromedriver.exe' # 크롬 드라이버 경로


options = webdriver.ChromeOptions()
options.add_argument('headless')    # 실행했을 때 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
options.add_argument('disable-gpu')    # GPU 사용 안함 --> headless 설정, gpu 사용 안함 : 속도 향상
options.add_argument('lang=ko_KR')    # 언어 설정
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko") # 봇 차단을 막기 위한 우회
driver = webdriver.Chrome(chromedriver, options=options) #  옵션 적용

driver.implicitly_wait(100) # 위 설정을 적용하는 데 시간이 걸리기 때문에 기다려주기 (최대 100초를 기다리되, 중간에 완료되면 바로 넘어감)



driver.get("http://webtoon.daum.net/#sort=free&genre=&tab=finish") # driver.get : () 링크로 접속 --> 다음 완결 웹툰 페이지 접속

data_name = driver.find_elements_by_class_name('tit_wt') # 웹툰 이름 크롤링
data_link = driver.find_elements_by_class_name('link_wt') # 웹툰 경로 크롤링
data_author = driver.find_elements_by_class_name('txt_info') # 웹툰 작가 크롤링
        # 무료 완결 웹툰에서 크롤링을 하면 마지막에 연재웹툰 1~5위 완결웹툰 1~5위 링크가 추가로 나옴
        # --> 링크가 이름보다 10개 많음(다 무료 완결 아니니 신경 X)

import csv 
f = open("write_temp.csv","w", encoding = "utf-8") # write.csv 파일 열기 (파일이 없어도 알아서 만들어줌)
                                              # 파일 경로는 현재 파이썬 파일(crawling_daumWebtoon) 경로와 같음
wr = csv.writer(f) # csv 파일을 수정할 것을 선언
    
for i in range(0, len(data_name)):# 반복의 기준은 완결 웹툰 이름 (링크는 갯수가 다름)
    webtoonName = data_name[i].text # 태그의 내용 (<span> 내용 </span>)을 읽어올 땐 .text
    
    driver_temp = webdriver.Chrome(chromedriver, options=options) #  옵션 적용
    driver_temp.implicitly_wait(100)
    
    link = "http://m." + data_link[i].get_attribute('href')[7:] # 모바일 링크로 변경
    last_link = link.replace(" ", "") # 위 코드에서 생기는 공백 제거 ( http://m, 과 링크 사이에 공백이 생김)
    
    
    driver_temp.get(last_link) # 완결 웹툰 상세페이지 받아오기
    
    if data_name[i].text[0:2] == "성인": # 성인웹툰은 이름 크롤링시 웹툰 이름 앞에 성인이 붙음
        webtoonName = webtoonName[3:]
                                                 # --> 웹툰 이름 앞 두글자가 성인이면 성인웹툰으로 판단 후 제외

        driver_temp.find_element_by_class_name("link_login.link_klogin").click()

        driver_temp.find_element_by_name('email').send_keys(kakao_id)
        driver_temp.find_element_by_name('password').send_keys(kakao_password)
        driver_temp.find_element_by_class_name('btn_g.btn_confirm.submit').click()
        # print("login_end \n")
    
    webtoonGenre = "" #변수선언
    print(webtoonName) # 웹툰 이름 출력
    print("작가 : ", data_author[i].text) # 작가 출력

    webtoonStory = driver_temp.find_element_by_class_name('txt_story') # 스토리 받아오기
    print(webtoonStory.text) # 웹툰 스토리 출력
        
        
    webtoonGenre = driver_temp.find_element_by_class_name('txt_genre') # 장르 받아오기
    print("장르 : ", webtoonGenre.text) # 웹툰 장르 출력
        
    data_thumb = driver_temp.find_element_by_class_name('img_thumb') # 웹툰 썸네일 -> 모바일 웹툰 상세페이지에서 받으면 배경이 없음
    print("썸네일 : ", data_thumb.get_attribute('src')) # 썸네일링크 출력    
        
    import urllib
    import re
    
    url_image = data_thumb.get_attribute('src')
        
    webtoonName_pre = re.sub('[^\s0-9a-zA-Zㄱ-힗!@#$%*()]', '', str(webtoonName)) # 저장이 안되는 특수기호 제거
    save1 = "D:/thumb_daum/" + webtoonName_pre + ".png"  # 이미지를 저장 할 경로
                        # 주소 앞에 자신한테 맞는 주소로 변경 -> 파일 만들어져있지 않으면 오류남 (D 드라이브에 thumb 폴더가 있는 상태로 실행)


    hdr = {'User-Agent': 'Mozilla/5.0'} # 봇 차단 방지를 위한 헤더 추가 (파이어폭스로 설정)

    req = urllib.request.Request(url_image, headers = hdr); 
    file_temp = urllib.request.urlopen(req).read()

    file_save = open(save1,'wb')  # w : 쓰기, r : 읽기 , a : 더하기, wb : 바이너리로 쓰기
    file_save.write(file_temp)
    file_save.close()
    
        
    link = "http://m." + data_link[i].get_attribute('href')[7:] # 모바일 링크로 변경
    last_link = link.replace(" ", "") # 위 코드에서 생기는 공백 제거 ( http://m, 과 링크 사이에 공백이 생김)
    print("링크 : ", last_link) # 링크 출력
    
    print("") #구분을 위한 줄 띄우기
    
    
    wr.writerow([webtoonName,data_author[i].text,data_thumb.get_attribute('src'),webtoonGenre.text,webtoonStory.text,last_link])
 
    
    #csv_writer.writerow((data_name[i].text, data_author[i].text, img_thumb, txt_genre, txt_story, last_link))
    
    driver_temp.quit()

f.close()

driver.quit()